In [14]:
import numpy as np
from numpy import *
import matplotlib
from matplotlib import pyplot as plt
from tqdm.auto import tqdm
import requests
import pickle
import re
import itertools
import functools
import collections
import string
import time
from bs4 import BeautifulSoup
from sortedcontainers import SortedList # http://www.grantjenks.com/docs/sortedcontainers/sortedlist.html#sortedlist

import multiprocess
# with multiprocess.Pool(processes=8) as pool:
#   for i in pool.imap_unordered(f, a):
#     print(i)

from adventlib.common import *
# n, m, a = lines_as_matrix_nm(lines)
# a = lines_as_matrix(lines)
# a = lines_as_digit_matrix(lines)
# chunks = split_iter_by_start_indices(a, indices)
# chunks = split_iter_by_bools(a, bools)

from adventlib.point_2d import *
# from adventlib.point_3d import *

YEAR = 2022
DAY = int('24')

submit1, submit2 = generate_submits(YEAR, DAY)

while True:
  try:
    raw_input = get_input(YEAR, DAY)
    break
  except Exception as e:
    print(e)
    time.sleep(1)

In [36]:
lines, groups = linesplit("""
#.######
#>>.<^<#
#.<..<<#
#>v.><>#
#<^v^^>#
######.#
""")
lines

['#.######', '#>>.<^<#', '#.<..<<#', '#>v.><>#', '#<^v^^>#', '######.#']

In [46]:
lines, groups = linesplit(raw_input, verbose=True)

27 lines
1 groups
>>>
#.##################################################################################################...
#<>v^<vv<v<><.^v><<.>v><<^^<v..<>.v^>^v<^><vv>.>^v>^>>v^v>><^><><<^><^>.^>^<vv>v>^>>^<^v.^<^v<.><.v>...
#<>.vv^^vv<^v.^><^v<>>.<<<.^<^<>v<v<<>v>^vv..<<.^>^>>>.<>>^.^<^<<<^<>vv>.<<><^>.^>.vv^^v..v^<v^<vv.^...
...


In [47]:
n, m, a = lines_as_matrix_nm(lines)

In [48]:
a

array([['#', '.', '#', ..., '#', '#', '#'],
       ['#', '<', '>', ..., 'v', '>', '#'],
       ['#', '<', '>', ..., '^', '>', '#'],
       ...,
       ['#', '>', '^', ..., '^', '<', '#'],
       ['#', '>', '>', ..., '>', '>', '#'],
       ['#', '#', '#', ..., '#', '.', '#']], dtype='<U1')

In [49]:
dirs = {
  '>': Point(0, 1),
  '<': Point(0, -1),
  'v': Point(1, 0),
  '^': Point(-1, 0),
}

In [50]:
blizzards = []
for i in range(n):
  for j in range(m):
    if a[i, j] != '#' and a[i, j] != '.':
      p = Point(i, j)
      blizzards.append((p, dirs[a[i, j]]))
blizzards

[([+1, +1], [+0, -1]),
 ([+1, +2], [+0, +1]),
 ([+1, +3], [+1, +0]),
 ([+1, +4], [-1, +0]),
 ([+1, +5], [+0, -1]),
 ([+1, +6], [+1, +0]),
 ([+1, +7], [+1, +0]),
 ([+1, +8], [+0, -1]),
 ([+1, +9], [+1, +0]),
 ([+1, +10], [+0, -1]),
 ([+1, +11], [+0, +1]),
 ([+1, +12], [+0, -1]),
 ([+1, +14], [-1, +0]),
 ([+1, +15], [+1, +0]),
 ([+1, +16], [+0, +1]),
 ([+1, +17], [+0, -1]),
 ([+1, +18], [+0, -1]),
 ([+1, +20], [+0, +1]),
 ([+1, +21], [+1, +0]),
 ([+1, +22], [+0, +1]),
 ([+1, +23], [+0, -1]),
 ([+1, +24], [+0, -1]),
 ([+1, +25], [-1, +0]),
 ([+1, +26], [-1, +0]),
 ([+1, +27], [+0, -1]),
 ([+1, +28], [+1, +0]),
 ([+1, +31], [+0, -1]),
 ([+1, +32], [+0, +1]),
 ([+1, +34], [+1, +0]),
 ([+1, +35], [-1, +0]),
 ([+1, +36], [+0, +1]),
 ([+1, +37], [-1, +0]),
 ([+1, +38], [+1, +0]),
 ([+1, +39], [+0, -1]),
 ([+1, +40], [-1, +0]),
 ([+1, +41], [+0, +1]),
 ([+1, +42], [+0, -1]),
 ([+1, +43], [+1, +0]),
 ([+1, +44], [+1, +0]),
 ([+1, +45], [+0, +1]),
 ([+1, +47], [+0, +1]),
 ([+1, +48], [-1, +0]),
 

In [51]:
def blizstep(blizzards):
  ans = []
  for p, dir in blizzards:
    pp = p + dir
    if pp.x == 0:
      pp = Point(n - 2, pp.y)
    elif pp.x == n - 1:
      pp = Point(1, pp.y)
    elif pp.y == 0:
      pp = Point(pp.x, m - 2)
    elif pp.y == m - 1:
      pp = Point(pp.x, 1)
    ans.append((pp, dir))
  return ans

In [52]:
q = collections.deque([(Point(0, 1), 0)])
exit = Point(n - 1, m - 2)
distance = {
  q[0]: 0,
}
prev = {}

blizz_at_time = {
  0: (blizzards, set([i for i, j in blizzards])),
}

while q:
  p, time = q.popleft()
  if time + 1 not in blizz_at_time:
    nextbliz = blizstep(blizz_at_time[time][0])
    nextset = set([i for i, j in nextbliz])
    blizz_at_time[time + 1] = (nextbliz, nextset)
    
  nextblizset = blizz_at_time[time + 1][1]
  
  for dir in DIRS4:
    pp = p + dir
    
    if a[pp.x, pp.y] == '#' or pp.x < 0 or pp.x >= n or pp.y < 0 or pp.y >= m:
      continue
    if pp in nextblizset or (pp, time + 1) in distance:
      continue
    
    distance[(pp, time + 1)] = distance[(p, time)] + 1
    prev[(pp, time + 1)] = (p, time)
    q.append((pp, time + 1))
    
    if pp == exit:
      print(pp, time + 1)
      break
  
  if (p, time + 1) not in distance and p not in nextblizset:
    distance[(p, time + 1)] = distance[(p, time)] + 1
    prev[(p, time + 1)] = (p, time)
    q.append((p, time + 1))

[+26, +120] 295


IndexError: index 27 is out of bounds for axis 0 with size 27

In [ ]:
p = (Point(5, 6), 17)

while True:
  print(p)
  if p in prev:
    p = prev[p]
  else:
    break

In [53]:
submit1(295)

Submitting 295...


True

In [54]:
q = collections.deque([(Point(0, 1), 0)])
exit = Point(n - 1, m - 2)
distance = {
  q[0]: 0,
}
prev = {}

blizz_at_time = {
  0: (blizzards, set([i for i, j in blizzards])),
}

while q:
  p, time = q.popleft()
  if time + 1 not in blizz_at_time:
    nextbliz = blizstep(blizz_at_time[time][0])
    nextset = set([i for i, j in nextbliz])
    blizz_at_time[time + 1] = (nextbliz, nextset)
    
  nextblizset = blizz_at_time[time + 1][1]
  
  for dir in DIRS4:
    pp = p + dir
    
    if a[pp.x, pp.y] == '#' or pp.x < 0 or pp.x >= n or pp.y < 0 or pp.y >= m:
      continue
    if pp in nextblizset or (pp, time + 1) in distance:
      continue
    
    distance[(pp, time + 1)] = distance[(p, time)] + 1
    prev[(pp, time + 1)] = (p, time)
    q.append((pp, time + 1))
    
    if pp == exit:
      print(pp, time + 1)
      break
  
  if (p, time + 1) not in distance and p not in nextblizset:
    distance[(p, time + 1)] = distance[(p, time)] + 1
    prev[(p, time + 1)] = (p, time)
    q.append((p, time + 1))

[+26, +120] 295


IndexError: index 27 is out of bounds for axis 0 with size 27

In [60]:
q = collections.deque([(Point(0, 1), 556)])
exit = Point(n - 1, m - 2)
distance = {
  q[0]: 0,
}
prev = {}

while q:
  p, time = q.popleft()
  if time + 1 not in blizz_at_time:
    nextbliz = blizstep(blizz_at_time[time][0])
    nextset = set([i for i, j in nextbliz])
    blizz_at_time[time + 1] = (nextbliz, nextset)
    
  nextblizset = blizz_at_time[time + 1][1]
  
  for dir in DIRS4:
    pp = p + dir
    
    if pp.x < 0 or pp.x >= n or pp.y < 0 or pp.y >= m or a[pp.x, pp.y] == '#':
      continue
    if pp in nextblizset or (pp, time + 1) in distance:
      continue
    
    distance[(pp, time + 1)] = distance[(p, time)] + 1
    prev[(pp, time + 1)] = (p, time)
    q.append((pp, time + 1))
    
    if pp == exit:
      print(pp, time + 1)
      print(len(q))
      raise Exception()
  
  if (p, time + 1) not in distance and p not in nextblizset:
    distance[(p, time + 1)] = distance[(p, time)] + 1
    prev[(p, time + 1)] = (p, time)
    q.append((p, time + 1))

[+26, +120] 851
888


Exception: 

In [58]:
submit2(851)

Submitting 851...


True